<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/sarcasm_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import numpy as np
import urllib

In [3]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f35c7fbac20>)

In [4]:
vocab_size = 10000
embedding_dim = 256
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

sentences = []
labels = []

In [5]:
import pandas as pd

df = pd.read_json('sarcasm.json')


In [9]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [10]:
df.sort_values(by="is_sarcastic", ascending=False)

,article_link,headline,is_sarcastic
21954,https://local.theonion.com/area-mans-biggest-a...,area man's biggest accomplishment not ever kil...,1
10347,https://politics.theonion.com/melania-trump-ho...,melania trump hosts state dinner in stunning b...,1
20977,https://local.theonion.com/opening-soda-bottle...,opening soda bottle inadvertently makes man loser,1
10326,https://entertainment.theonion.com/david-blain...,david blaine starves self of attention for 33 ...,1
10327,https://local.theonion.com/neighbor-bragging-a...,neighbor bragging about 20-pound box he fedexed,1
...,...,...,...
11465,https://www.huffingtonpost.com/entry/medical-m...,new proposal could make it easier to research ...,0
11462,https://www.huffingtonpost.com/entry/trump-nat...,trump's national security transition is 'a mes...,0
11461,https://www.huffingtonpost.com/entry/how-imani...,how imani boyette's love for basketball helped...,0
11460,https://www.huffingtonpost.com/entry/barack-ob...,barack obama is fourth president to put americ...,0


In [6]:
df = df[df['is_sarcastic'] > 0]

In [7]:
max_length = int(df['headline'].str.len().max()) + 1

In [8]:
sentences=list(df['headline'])


In [14]:
len(sentences)

11724

In [29]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer


In [10]:
from tensorflow.keras.layers import TextVectorization

In [41]:
text_vectorization = TextVectorization (ragged=True, output_mode="int")

In [42]:
text_vectorization.adapt(sentences)

In [45]:
vec_sentences = text_vectorization(sentences)

In [48]:
vec_sentences[12]

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([7009, 4350,  403, 6112, 2300,   54,   26, 1376])>

In [51]:
type(vec_sentences)

tensorflow.python.ops.ragged.ragged_tensor.RaggedTensor

In [53]:
padded = vec_sentences.to_tensor(0)

In [55]:
final_tensor = tf.reverse(padded, [-1])

In [56]:
final_tensor[0]

<tf.Tensor: shape=(39,), dtype=int64, numpy=
array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 8070,    2,   42,   53,  124,  171, 2675, 1079,
       2415, 1075, 1122,    2,  499,   96])>

In [54]:
padded[0]

<tf.Tensor: shape=(39,), dtype=int64, numpy=
array([  96,  499,    2, 1122, 1075, 2415, 1079, 2675,  171,  124,   53,
         42,    2, 8070,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])>

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(32)),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 120)               7800      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 36,465
Trainable params: 36,465
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("1.h5", save_best_only=True)
]

In [ ]:
history = model.fit(vec_train_sentences, np.array(training_labels),
                    validation_data = (vec_val_sentences, np.array(val_labels)),
                    callbacks=callbacks,
                    epochs=32)

Epoch 1/32
585/585 [==============================] - 10s 17ms/step - loss: 0.3622 - accuracy: 0.8370 - val_loss: 0.3749 - val_accuracy: 0.8298
Epoch 2/32
585/585 [==============================] - 9s 16ms/step - loss: 0.3350 - accuracy: 0.8490 - val_loss: 0.3774 - val_accuracy: 0.8303
Epoch 3/32
585/585 [==============================] - 9s 15ms/step - loss: 0.3141 - accuracy: 0.8611 - val_loss: 0.3699 - val_accuracy: 0.8320
Epoch 4/32
585/585 [==============================] - 9s 15ms/step - loss: 0.3001 - accuracy: 0.8697 - val_loss: 0.3820 - val_accuracy: 0.8300
Epoch 5/32
585/585 [==============================] - 8s 14ms/step - loss: 0.2897 - accuracy: 0.8722 - val_loss: 0.3836 - val_accuracy: 0.8310
Epoch 6/32
585/585 [==============================] - 9s 15ms/step - loss: 0.2809 - accuracy: 0.8784 - val_loss: 0.4023 - val_accuracy: 0.8233
Epoch 7/32
585/585 [==============================] - 8s 14ms/step - loss: 0.2721 - accuracy: 0.8820 - val_loss: 0.4109 - val_accuracy: 0.814

In [ ]:
test_sentences[2]

'10 secrets of irresistible people'

In [ ]:
print(text_vectorization(test_sentences[2]))

tf.Tensor(
[152   1   3   1  64   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0], shape=(120,), dtype=int64)


In [ ]:
print(model.predict(text_vectorization(["this is example of laziness", "My neighbors listen to good music whether they like it or not"])))

1/1 [==============================] - 0s 124ms/step
[[0.00130517]
 [0.99801743]]
